## Notebook for extracting neighborhoods in Toronto from Wiki Page and adding coordinates

### Scrape Wikipedia page

Installing prerequisites

In [1]:
#importing required libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

In [2]:
#installing BeautifulSoup library to help scrape WikiPage (if not installed before)
!pip install beautifulsoup4
#installing lxml parser for BeautifulSoup to use (if not installed before)
!pip install lxml
#installing requests to work with URLs (if not installed before)
!pip install requests

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
#importing BeautifulSoup and requests
from bs4 import BeautifulSoup
import requests

Work with URL and load data to dataframe

In [4]:
#load WikiPage with list of PostalCodes for Toronto
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [5]:
#cut a table from page to work with
table = soup.find('table', class_='wikitable sortable')

Constructing dataframe

In [6]:
#create columns out of headers in table
column_names = []
for header in table.find_all('th'):
    column_names.append(header.text.strip())
#create new dataframe with extracted columns
df = pd.DataFrame(columns=column_names)

In [7]:
#extract rows from table based on <tr> tag
rows = table.find_all('tr')
for row in rows[1:]: #skipping zero element as it's header
    cols = row.find_all('td')
    if cols[1].text.strip() == 'Not assigned': #skipping lines with not assigned borough
        continue
    elif cols[2].text.strip() == 'Not assigned': #if neighbourhood not assigned make it equal to borough
        df = df.append([{'Postcode':cols[0].text.strip(), 'Borough':cols[1].text.strip(), 'Neighbourhood':cols[1].text.strip()}], ignore_index=True)
    else:
        df = df.append([{'Postcode':cols[0].text.strip(), 'Borough':cols[1].text.strip(), 'Neighbourhood':cols[2].text.strip()}], ignore_index=True) 

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
df.shape

(212, 3)

Combining neighbourhoods with the same postcode and borough

In [10]:
df = df.groupby(['Postcode','Borough'], as_index=False, sort=False).agg(','.join)

In [11]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [12]:
df.shape

(103, 3)

Tried to get coord through Geocoder API but got the Null all the time. When using loop, it runs infinitely because getting Null all the time as well.

In [13]:
#!pip install geocoder
#import geocoder
#g = geocoder.google('M2H, Toronto, Ontario')
#print(g.latlng)
#None here all the time, same happens with while loop

Because of failed attempts with Geocoder, I have to use provided csv file with coordinates

In [14]:
#reading csv and exploring
df1 = pd.read_csv('https://cocl.us/Geospatial_data')
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Renaming column name Postal Code to Postcode so it's easier to merge

In [15]:
df1.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df1.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging coordinates from df1(that was created from csv) with our df neighbourhood dataframe

In [16]:
df2 = pd.merge(df, df1, on='Postcode')
df2.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [17]:
df2.shape

(103, 5)